In [1]:
// 1. Installation
// https://github.com/helgeho/ArchiveSpark/blob/master/notebooks/Selected_Title-and-Text.ipynb

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._


// prepare session for creating data frames
// https://sparkbyexamples.com/spark/different-ways-to-create-a-spark-dataframe/

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

val spark2 = spark
val session2 = spark2.newSession

import spark2.implicits._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"


// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
// 2. get all data for research year
val r1 = r.filter(r => r.timestamp.startsWith("1996"))

In [3]:
r1.peekJson

{
    "record" : {
        "redirectUrl" : "-",
        "timestamp" : "19961230003113",
        "digest" : "SEIFFLPZ24OLR4RJPFQIA7VXM6AZG25Y",
        "originalUrl" : "http://studio-net.com.pl:80/",
        "surtUrl" : "pl,com,studio-net)/",
        "mime" : "text/html",
        "compressedSize" : 1405,
        "meta" : "-",
        "status" : 200
    }
}

In [7]:
val m1 = r1.map(f=> (f.originalUrl,f.surtUrl,f.redirectUrl,f.digest,f.status))
val df = session.createDataFrame(m1).toDF("originalUrl","surtUrl","redirectUrl","digest","status").cache()

In [8]:
df.cache().show()

+--------------------+--------------------+-----------+--------------------+------+
|         originalUrl|             surtUrl|redirectUrl|              digest|status|
+--------------------+--------------------+-----------+--------------------+------+
|http://studio-net...| pl,com,studio-net)/|          -|SEIFFLPZ24OLR4RJP...|   200|
|http://studio-net...|pl,com,studio-net...|          -|J7RVGRQDPWWKUZIHO...|   200|
|http://studio-net...|pl,com,studio-net...|          -|CHZGLOERIPVY7PJKF...|   200|
|http://studio-net...|pl,com,studio-net...|          -|4IEAZP7DIEV4BFZA6...|   200|
|http://studio-net...|pl,com,studio-net...|          -|LRWGP7FT2X5LK5MGP...|   200|
|http://peclet.sun...|pl,com,suntech,pe...|          -|JXPE6KEXM43S6O5UO...|   200|
|http://peclet.sun...|pl,com,suntech,pe...|          -|JXPE6KEXM43S6O5UO...|   200|
|http://peclet.sun...|pl,com,suntech,pe...|          -|JXPE6KEXM43S6O5UO...|   200|
|http://peclet.sun...|pl,com,suntech,pe...|          -|FMWX5T2OZBC3ZRCTL...|

In [10]:
// distinct for digest
df.select(countDistinct("digest")).show(false)

+----------------------+
|count(DISTINCT digest)|
+----------------------+
|61678                 |
+----------------------+



In [23]:
df.select(countDistinct("originalUrl")).show(false)

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|72229                      |
+---------------------------+



In [22]:
df.select(countDistinct("surtUrl")).show(false)

+-----------------------+
|count(DISTINCT surtUrl)|
+-----------------------+
|71303                  |
+-----------------------+



In [17]:
val df2 = df.groupBy("status").count()

In [18]:
df2.show()

+------+------+
|status| count|
+------+------+
|   500|    32|
|   200|161192|
|   302|    94|
+------+------+



In [19]:
val df3 = df.groupBy("redirectUrl").count()

In [20]:
df3.show()

+-----------+------+
|redirectUrl| count|
+-----------+------+
|          -|160936|
|          S|   382|
+-----------+------+



In [21]:
r1.count()

161318